In [126]:
import json

with open('task-1.advising.train.json', 'rb') as f :
    json_data = json.load(f)
    
with open('task-1.advising.dev.json', 'rb') as f :
    dev_data = json.load(f)

In [120]:
json_data = json_data[:10]
dev_data = dev_data[:1]

In [127]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_data = dict()
train_data['input_ids'] = list()
train_data['segment_ids'] = list()
train_data['attention_masks'] = list()

for data in json_data :
    sentence_a = list()
    instance_input = list()
    instance_seg = list()
    instance_attn = list()
    for msg in data['messages-so-far'] :
        sentence_a.append(msg['utterance'])
    sentence_a = ' '.join(sentence_a)
    for option in data['options-for-next']:
        candidate = tokenizer.encode_plus(text=sentence_a, text_pair=option['utterance'], max_length = 120, pad_to_max_length = True)        
        instance_input.append(candidate['input_ids'])
        instance_seg.append(candidate['token_type_ids'])
        instance_attn.append(candidate['attention_mask'])
    train_data['input_ids'].append(instance_input)
    train_data['segment_ids'].append(instance_seg)
    train_data['attention_masks'].append(instance_attn)

In [128]:
train_data['labels'] = list()
for data in json_data :
    label = [0 for i in range(100)]
    if len(data['options-for-correct-answers']) < 1 :
        train_data['labels'].append(label)
        continue
    for idx, option in enumerate(data['options-for-next']):
        if data['options-for-correct-answers'][0]['utterance'] == option['utterance'] :
            label[idx] += 1
    train_data['labels'].append(label)

In [129]:
val_data = dict()
val_data['input_ids'] = list()
val_data['segment_ids'] = list()
val_data['attention_masks'] = list()

for data in dev_data :
    sentence_a = list()
    instance_input = list()
    instance_seg = list()
    instance_attn = list()
    for msg in data['messages-so-far'] :
        sentence_a.append(msg['utterance'])
    sentence_a = ' '.join(sentence_a)
    for option in data['options-for-next']:
        candidate = tokenizer.encode_plus(text=sentence_a, text_pair=option['utterance'], max_length = 120, pad_to_max_length = True)        
        instance_input.append(candidate['input_ids'])
        instance_seg.append(candidate['token_type_ids'])
        instance_attn.append(candidate['attention_mask'])
    val_data['input_ids'].append(instance_input)
    val_data['segment_ids'].append(instance_seg)
    val_data['attention_masks'].append(instance_attn)

In [130]:
val_data['labels'] = list()
for data in dev_data :
    label = [0 for i in range(100)]
    if len(data['options-for-correct-answers']) < 1 :
        val_data['labels'].append(label)
        continue
    for idx, option in enumerate(data['options-for-next']):
        if data['options-for-correct-answers'][0]['utterance'] == option['utterance'] :
            label[idx] += 1
    val_data['labels'].append(label)

In [131]:
import pickle

with open('preprocessed_advising_large.pkl', 'wb') as f :
    pickle.dump((train_data, val_data), f)

In [117]:
tokens = torch.tensor(train_data['input_ids'])
segs = torch.tensor(train_data['segment_ids'])
attn = torch.tensor(train_data['attention_masks'])
labl = torch.tensor(train_data['labels'])

In [118]:
print(tokens.shape, segs.shape, attn.shape, labl.shape)

torch.Size([100, 100, 120]) torch.Size([100, 100, 120]) torch.Size([100, 100, 120]) torch.Size([100, 100])


In [139]:
a = [(1,2), (2,4)]

In [140]:
a.sort(key=lambda element:element[1], reverse = True)

In [141]:
a

[(2, 4), (1, 2)]

In [143]:
from torch import nn
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)

In [144]:
input

tensor([[ 2.2335, -0.6044,  1.4533],
        [ 0.0133,  0.6811,  1.1492]])

In [145]:
output

tensor([[0.6593, 0.0386, 0.3021],
        [0.1649, 0.3216, 0.5135]])

In [18]:
import torch
from torch import nn

m = nn.Softmax(dim=0)
input = torch.randn(3, 1)
output = m(input)

In [19]:
input.shape

torch.Size([3, 1])

In [20]:
output

tensor([[0.6708],
        [0.2931],
        [0.0360]])

In [22]:
output.squeeze(1).shape

torch.Size([3])